# Removing duplicate tuples from the tables

In [5]:
import numpy as np
import pandas

# Books Amazon
amazon = pandas.read_csv('Data/books_amazon_output.csv')
unique_amazon = amazon.drop_duplicates(subset='ISBN-10', keep='first', inplace=False)
unique_amazon.to_csv('Data/books_amazon_unique.csv', index = False)

# Books Millions
millions= pandas.read_csv('Data/books_millions_output.csv')
unique_millions = millions.drop_duplicates(subset='ISBN-10', keep='first', inplace=False)
unique_millions.to_csv('Data/books_millions_unique.csv', index = False)

# Cleaning and preprocessing the data

-> Add column 'ID'. <br/>
-> Ensure fields containing comma must be double quoted.

In [6]:
import pandas
import numpy as np
import csv


def clean(inputfile, outputfile):
    ID = 'ID'
    NAME = 'Name'
    CATEGORY = 'Category'
    AUTHOR = 'Author'
    PRICE = 'Price'
    SERIES = 'Series'
    PAGES = 'Pages'
    PUBLISHER = 'Publisher'
    DATE = 'Date'
    LANGUAGE = 'Language'
    ISBN_10 = 'ISBN_10'
    ISBN_13 = 'ISBN_13'
    DIMENSIONS = 'Dimensions'
    WEIGHT = 'Weight'
    COMMA = ','
    NEW_LINE = '\n'



    df = pandas.read_csv(inputfile)
    df = df.fillna('')
    df.insert(0, 'ID', 1)
    X = df.as_matrix()
    for i in range(0,len(X)):
        X[i][0] = i +1
        for j in range(0,len(X[0])):
            if type(X[i][j]) == str and "#" in X[i][j]:
                X[i][j] = X[i][j].replace("#",",")
                X[i][j] = '"' + X[i][j] + '"'

    header = ID+COMMA+NAME+COMMA+CATEGORY+COMMA+AUTHOR+COMMA+PRICE+COMMA+SERIES+COMMA+PAGES+COMMA+PUBLISHER+COMMA+DATE+COMMA+LANGUAGE+COMMA+ISBN_10+COMMA+ISBN_13+COMMA+DIMENSIONS+COMMA+WEIGHT+NEW_LINE
    myFile = open(outputfile, 'w')
    myFile.write(header)
    with myFile:
        writer = csv.writer(myFile)
        writer.writerows(X)

    myFile.close()


clean('Data/books_amazon_unique.csv','Data/books_amazon_clean.csv')
clean('Data/books_millions_unique.csv','Data/books_millions_clean.csv')

# Blocking

Implemented Attribute Equivalence, Overlap and Rule-bvased blockers. <br/>
Selected Rule-based blocker with following rules: <br/>
1. Jaccard measure on 'Author' <br/>
2. Cosine measure on 'Name' <br/>
3. Mel measure on 'Author' 
<br/><br/>
Created candidate tuple pairs in table C, and verified the blocking step using table D.

In [7]:
import py_entitymatching as em
import numpy
from numpy import block
import pandas as pd

#*************** Load data into dataframe*************************
A=em.read_csv_metadata('Data/books_amazon_clean.csv',key='ID')
B=em.read_csv_metadata('Data/books_millions_clean.csv',key='ID')

# Test dataframe
#A_test = A.head(10)
#B_test = B.head(10)

# Blocker 1 AttrEquivalenceBlocker
# b1=em.AttrEquivalenceBlocker()
# C1=b1.block_tables(A, B, "Weight", "Weight", l_output_attrs=['Name'], r_output_attrs=['Name'])

# Blocker 2 Overlap
# b2=em.OverlapBlocker()
# C2=b2.block_tables(A,B,'Name','Name',word_level=True,overlap_size=2,l_output_attrs=['Name'], r_output_attrs=['Name'],rem_stop_words= True)

# Blocker 3 rule based blocker
b4=em.RuleBasedBlocker()
#****************** Change feature datatype***************************
a_types = em.get_attr_types(A)
b_types = em.get_attr_types(B)
b_types['Name']= a_types['Name']

#******************Create Blocker ***********************************
# block_f = em.get_features_for_blocking(A,B,validate_inferred_attr_types=False)
block_c = em.get_attr_corres(A,B)
block_t = em.get_tokenizers_for_blocking()
block_s = em.get_sim_funs_for_blocking()
block_f=em.get_features(A,B,a_types,b_types,block_c,block_t,block_s)

#******************************** Add Rules *************************
b4.add_rule(['Author_Author_jac_dlm_dc0_dlm_dc0(ltuple, rtuple) < 0.2'], block_f)
b4.add_rule([' Name_Name_cos_dlm_dc0_dlm_dc0(ltuple, rtuple) < 0.3'], block_f)
b4.add_rule(['Author_Author_mel(ltuple, rtuple) < 0.5'], block_f)
# Unused Rules
# b4.add_rule([' Publisher_Publisher_jac_dlm_dc0_dlm_dc0(ltuple, rtuple) < 0.3'], block_f)
# b4.add_rule(['name_name_lev_sim(ltuple, rtuple) < 0.8'],block_f)
# b4.add_rule(['Category_Category_lev_sim(ltuple, rtuple) < 0.5'], block_f)
# b4.add_rule(['Category_Category_jac_dlm_dc0_dlm_dc0(ltuple, rtuple) < 0.5'], block_f)

column_names = ['ID','Name', 'Category','Author','Price','Series','Pages','Publisher','Date','Language','ISBN_10','ISBN_13','Dimensions','Weight']
#******************* Blocking step**********************
C = b4.block_tables(A, B, l_output_attrs=column_names, r_output_attrs=column_names)
C.to_csv('Data/C.csv', index = False)

#**************************** Debug Blocking******************************
D = em.debug_blocker(C, A, B)
D.to_csv('Data/D.csv', index = False)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:02
0%  100%
[##] | ETA: 00:00:00

Finding pairs with missing value...



Total time elapsed: 00:00:00
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:19


# Sampling

Sampled 500 tuple pairs from table C for matching. <br/>
Labeled these 500 tuple pairs manually (1=true match, else 0)

In [4]:
import pandas as pd
import py_entitymatching as em

A=em.read_csv_metadata('Data/books_amazon_clean.csv',key='ID')
B=em.read_csv_metadata('Data/books_millions_clean.csv',key='ID')
C=em.read_csv_metadata('Data/C.csv',key='_id', fk_ltable = 'ltable_ID', fk_rtable = 'rtable_ID', ltable = A, rtable = B)

S = em.sample_table(C, 500)
G = S
# Labeled the sample tuple pairs manually
G.to_csv('Data/G.csv', index = False)

# Matching

Created training set I and test set J  using the labeled tuple pairs.  <br/>
Set I and set J contain 250 tuple pairs each. <br/>
Implemented 6 learning based matchers using cross-validation.<br/>
Dropped ISBN related features. <br/>
Based on performance on Training set I, selected Random Forest as the best matcher.<br/>
Calculated the performance of all 6 matchers on the test set J.

In [9]:
import pandas as pd
import py_entitymatching as em

seed = 0
#******************************** Read Labeled Sample Data ************************************
A=em.read_csv_metadata('Data/books_amazon_clean.csv',key='ID')
B=em.read_csv_metadata('Data/books_millions_clean.csv',key='ID')
G=em.read_csv_metadata('Data/G.csv',key='_id', fk_ltable = 'ltable_ID', fk_rtable = 'rtable_ID', ltable = A, rtable = B)

#******************************** Split into Train(I) and Test(J) data*************************
IJ = em.split_train_test(G,train_proportion=0.5, random_state=0)
I = IJ['train']
J = IJ['test']
I.to_csv('Data/I.csv', index = False)
J.to_csv('Data/J.csv', index = False)

#******************************** Instantiating the Learning-Based Matchers*********************

dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name = 'Naive Bayes')

#******************************** Creating Features *********************************************

#************* Change feature datatype*****************
a_types = em.get_attr_types(A)
b_types = em.get_attr_types(B)
b_types['Name']= a_types['Name']

match_c = em.get_attr_corres(A,B)
match_t = em.get_tokenizers_for_blocking()
match_s = em.get_sim_funs_for_blocking()

F = em.get_features(A,B,a_types,b_types,match_c, match_t, match_s,)

#******************************** Drop Attributes: ISBN_10 and ISBN_13  ************************

drop_list_index = [47,48,49,50,51,52]
F =F.drop(drop_list_index)

#********************************  Extracting Feature Vectors **********************************
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I,feature_table= F,attrs_after='label',show_progress=False)


#********************************Checking / Impute for missing values **************************
# Impute feature vectors with the mean of the column values.
H = em.impute_table(H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], strategy='mean')


#********************************Select the best ML matcher ************************************

result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,target_attr='label', metric_to_select_matcher='f1', random_state=0)


#******************************** Display Results **********************************************
print(result['cv_stats'])

print(result['drill_down_cv_stats']['precision'])
print(result['drill_down_cv_stats']['recall'])
print(result['drill_down_cv_stats']['f1'])


#******************************** Compute Accuracy of Test Set J********************************

#************Function to calculate accuracy ********************
def compute_accuracy_J(matcher,return_probs_arg, H, J):
    # Train using feature vectors from I
    matcher.fit(table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], target_attr='label')

    # Convert J into a set of feature vectors using F
    L = em.extract_feature_vecs(J, feature_table=F,
                                attrs_after='label', show_progress=False)
    # Impute L
    L = em.impute_table(L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], strategy='mean')

    # Predict on L
    predictions = matcher.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], append=True,
                                  target_attr='predicted', inplace=False, return_probs=return_probs_arg, probs_attr='proba')
    # print(predictions.head())

    # Evaluate the predictions
    eval_result = em.eval_matches(predictions, 'label', 'predicted')
    em.print_eval_summary(eval_result)



#******************************** Compute accuracy for each ML model ********************************

dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name = 'Naive Bayes')

all_matchers = [dt,svm,rf,lg,ln, nb]
return_probs = [True,False,True,True,True, True]
ML_model = ['Decision Tree', 'SVM' ,'Random Forest' ,'Logistic Regression' ,'Linear Regression', 'Naive Bayes']
print("\n\n Computing results on set J after training on set I \n \n")
for i in range(0,len(all_matchers)):
    print("For " + ML_model[i])
    compute_accuracy_J(all_matchers[i],return_probs[i],H,J)
    print("**************************************************************")

        Matcher  Average precision  Average recall  Average f1
0  DecisionTree           0.928421        0.924786    0.918968
1            RF           0.966013        0.924786    0.941175
2           SVM           1.000000        0.199145    0.328333
3        LinReg           0.597421        0.838355    0.690269
4        LogReg           0.896667        0.882692    0.885463
5   Naive Bayes           0.878788        0.969231    0.915246
           Name  \
0  DecisionTree   
1            RF   
2           SVM   
3        LinReg   
4        LogReg   
5   Naive Bayes   

                                                                            Matcher  \
0          <py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7f6fa2523a10>   
1          <py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7f6fa2d616d0>   
2        <py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7f6fa2523cd0>   
3  <py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7f6fa23

Best matcher based on training set I: Random Forest <br/>
Performance on training set I: <br/>
Precision : 96.6%<br/>
Recall : 92.4% <br/>
F1 : 94.1% <br/><br/>
Performance of Random Forest on test set J:<br/>
Precision : 96.0% <br/>
Recall : 88.89% <br/>
F1 : 92.31%